In [17]:
import os
from IPython.display import clear_output
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
import rasterio as rio

In [18]:
# The sizes of the output shps
shps = [30,100,300,600,900,1200,1800]
# Create also .json files?
bool_JSON = True

In [19]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [20]:
df_Site = pd.read_excel(cwd + "//Site - With New (Coordinates Only).xlsx")
df_Site.head(5)

,Number,Site,Latitude,Longitude,Reference network
0,1,ATGE,52.466778,12.959778,HYPERNET
1,2,ATLAS-Mohammed V,33.406152,-5.103319,Other
2,3,AT-Mmg,47.316700,10.970300,FLOX
3,4,BASP,39.049139,-2.075917,HYPERNET
4,5,BE-Bra,51.307600,4.519900,FLOX-ICOS


### Batch create .shp files

In [21]:
for i in range(df_Site.shape[0]):
    # Read info
    temp_SiteName = df_Site['Site'][i]
    temp_Latitude = df_Site['Latitude'][i]
    temp_Longitude = df_Site['Longitude'][i]
    print(f"{temp_SiteName} starts! ")
    # Create gdf
    df_4326 = pd.DataFrame({
        "Site": [temp_SiteName],
        "Latitude": [temp_Latitude],
        "Longitude": [temp_Longitude]
    })
    gdf_4326 = gpd.GeoDataFrame(
        df_4326,
        geometry = gpd.points_from_xy(df_4326['Longitude'], df_4326['Latitude']),
        crs = "EPSG:4326"
    )
    # Get L1C raster file
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + temp_SiteName + "\\L1C"):
        for name in files:
            temp = os.path.join(path, name)
            if "IMG_DATA" in temp and temp[-3:] == 'jp2' and "B08" in temp:
                # print(temp)
                path_L1C_B08_raw = temp
    print("The path to B08 of L1C is " + path_L1C_B08_raw)
    # Get L2A raster file
    for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + temp_SiteName + "\\L2A"):
        for name in files:
            temp = os.path.join(path, name)
            if temp[-3:] == 'jp2'in temp and "10m" in temp and "B04" in temp :
                path_L2A_B04_raw = temp
    print("The path to B04 of L2A is " + path_L2A_B04_raw)
    # Read rasters
    image_L1C_B08 = rio.open(path_L1C_B08_raw)
    image_L2A_B04 = rio.open(path_L2A_B04_raw)
    # Check crs
    crs_L1C = image_L1C_B08.crs.data["init"].split(":")[1]
    crs_L2A = image_L2A_B04.crs.data["init"].split(":")[1]
    # In the case that L1C and L2A have different crs, give an error. But this shouldn't happen. 
    if crs_L2A != crs_L1C:
        print("The EPSG of L1C is " + crs_L1C)
        print("The EPSG of L2A is " + crs_L2A)
        print("Please make sure the downloaded L2A and L1C images have the same crs. ")
        raise SystemExit("Stop right there!")
    crs_Final = 'EPSG:' + crs_L1C
    print("The final crs is " + crs_Final)
    # Converting Lon-Lat to target crs!
    gdf_New = gdf_4326.copy()
    gdf_New = gdf_New.to_crs(crs_Final)
    # Assign the site to a pixel
    site_x = gdf_New.geometry.x.values[0]
    site_y = gdf_New.geometry.y.values[0]
    site_row, site_col = image_L2A_B04.index(site_x, site_y)
    site_pixel_x, site_pixel_y = image_L2A_B04.xy(site_row, site_col)
    # Calculate the "cardinal" distance
    for size in shps:
        side_length_half = size / 2
        if side_length_half % 2 == 0:
            # If the half of the side length is even, we need to add another 5 meters to make sure the pixels on the borders will not be omitted when we clip the raster images. 
            length_Cardinal = side_length_half + 5
        else:
            length_Cardinal = side_length_half
        site_x_left_New = site_pixel_x - length_Cardinal
        site_x_right_New = site_pixel_x + length_Cardinal
        site_y_top_New = site_pixel_y + length_Cardinal
        site_y_bottom_New = site_pixel_y - length_Cardinal
        # Create a bounding box
        shp_New = shp.box(site_x_left_New, site_y_bottom_New, site_x_right_New, site_y_top_New)
        # Create shapefile! 
        gdf_New = gpd.GeoDataFrame(
            pd.DataFrame({"0": ["0"]}),
            geometry=[shp_New],
            crs = crs_Final
        )
        gdf_New.to_file(cwd_Images_Processed + "\\" + temp_SiteName + "\\" + str(size) + "m.shp")
        if bool_JSON:
            gdf_New.to_file(cwd_Images_Processed + "\\" + temp_SiteName + "\\" + str(size) + "m.json", driver = "GeoJSON")
    print(f"{temp_SiteName} finishes! ")
    clear_output(wait=True)
print("The creation of all sites has finished successfully!")

The creation of all sites has finished successfully!
